In [73]:
#Import libraries
from pyhive import presto
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import h3
from shapely.geometry import Polygon, Point
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

In [92]:
#Import the sourcing datasets for hyderabad
df_all = pd.read_csv('sourcing_data_hyderabad.csv')
df_all.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,zone_name
0,634a737d65bd2f18f9d9c37d,1.009,8860a259bdfffff,17.451334,78.396278,Hyderabad,8860a259b9fffff,17.443066,78.387779,141653,14,1415,Link,14:00,6,20221015,dropped,NaN,zone1
1,634a73ec1b4dce7515feb1ac,0.238,8860a25939fffff,17.444968,78.447435,Hyderabad,8860a25939fffff,17.443622,78.446238,141844,14,1415,Link,14:00,6,20221015,customerCancelled,customer not reachable,zone2
2,634a743cf620a666d8e77e60,0.397,8860a25a35fffff,17.357098,78.447868,Hyderabad,8860a259bbfffff,17.441568,78.376694,142004,14,1415,Link,14:00,6,20221015,dropped,NaN,zone1
3,634a77ff86b453679bd4a264,0.486,8860a24a63fffff,17.443333,78.354346,Hyderabad,8860a24a29fffff,17.448180,78.348464,143607,14,1430,Link,14:59,6,20221015,customerCancelled,Customer asked to cancel,zone1
4,634a78461b4dce7515feb9de,2.219,8860a25983fffff,17.437529,78.389496,Hyderabad,8860a24a65fffff,17.447975,78.365540,143718,14,1430,Link,14:59,6,20221015,dropped,NaN,zone1


In [93]:
#Create a count column
df_all['count'] = 1

#Create a new column for cancel_reason to identify OCARA, COBRA and COBRM
df_all['new_cancel_reason'] = np.where(df_all['order_status'] == 'customerCancelled', 
    np.where(df_all['cancel_reason'] == 'order cancelled before rider accepted', 'COBRA',
    np.where(df_all['cancel_reason'] == 'Order cancelled before rider was mapped', 'COBRM', 'OCARA')), 'no_cancellation')

#Create a column to indicate final order status
df_all['final_order_status'] = np.where(df_all['order_status'] == 'customerCancelled', df_all['new_cancel_reason'], df_all['order_status'])
df_all.head()

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,zone_name,count,new_cancel_reason,final_order_status
0,634a737d65bd2f18f9d9c37d,1.009,8860a259bdfffff,17.451334,78.396278,Hyderabad,8860a259b9fffff,17.443066,78.387779,141653,14,1415,Link,14:00,6,20221015,dropped,NaN,zone1,1,no_cancellation,dropped
1,634a73ec1b4dce7515feb1ac,0.238,8860a25939fffff,17.444968,78.447435,Hyderabad,8860a25939fffff,17.443622,78.446238,141844,14,1415,Link,14:00,6,20221015,customerCancelled,customer not reachable,zone2,1,OCARA,OCARA
2,634a743cf620a666d8e77e60,0.397,8860a25a35fffff,17.357098,78.447868,Hyderabad,8860a259bbfffff,17.441568,78.376694,142004,14,1415,Link,14:00,6,20221015,dropped,NaN,zone1,1,no_cancellation,dropped
3,634a77ff86b453679bd4a264,0.486,8860a24a63fffff,17.443333,78.354346,Hyderabad,8860a24a29fffff,17.448180,78.348464,143607,14,1430,Link,14:59,6,20221015,customerCancelled,Customer asked to cancel,zone1,1,OCARA,OCARA
4,634a78461b4dce7515feb9de,2.219,8860a25983fffff,17.437529,78.389496,Hyderabad,8860a24a65fffff,17.447975,78.365540,143718,14,1430,Link,14:59,6,20221015,dropped,NaN,zone1,1,no_cancellation,dropped


In [94]:
#Create a list of hexes belonging to both of the batching areas
zone1_hexes = list(df_all[df_all['zone_name'] == 'zone1']['customer_location_hex_8'].unique())
zone2_hexes = list(df_all[df_all['zone_name'] == 'zone2']['customer_location_hex_8'].unique())

#First we will check the captain_hexes for orders that resulted in drop
df_all['captain_customer_same_area'] = np.where(df_all['zone_name'] == 'zone1',
 np.where(df_all['captain_location_hex_8'].isin(zone1_hexes), 'sourced_from_batching_area', 'sourced_from_outside_batching_area'),
 np.where(df_all['captain_location_hex_8'].isin(zone2_hexes), 'sourced_from_batching_area', 'sourced_from_outside_batching_area'))

#Check the number of orders serviced within the batching area vs outside the batching area by zone
sourced_zone = pd.pivot_table(data = df_all, index = ['zone_name', 'yyyymmdd', 'hour', 'final_order_status'], columns = ['captain_customer_same_area'], 
values = 'count', aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Create a column for total rides
sourced_zone['total'] = sourced_zone[['sourced_from_batching_area', 'sourced_from_outside_batching_area']].sum(axis = 1)

#Calculate the percentage of each order_status
sourcing_zone = ['sourced_from_batching_area', 'sourced_from_outside_batching_area']
for zone in sourcing_zone:
  sourced_zone['{}_%'.format(zone)] = round(sourced_zone[zone]/sourced_zone['total'], 4)
sourced_zone.head()

,zone_name,yyyymmdd,hour,final_order_status,sourced_from_batching_area,sourced_from_outside_batching_area,total,sourced_from_batching_area_%,sourced_from_outside_batching_area_%
0,zone1,20220919,0,COBRA,33,1,34,0.9706,0.0294
1,zone1,20220919,0,OCARA,64,8,72,0.8889,0.1111
2,zone1,20220919,0,dropped,100,73,173,0.5780,0.4220
3,zone1,20220919,1,COBRA,13,0,13,1.0000,0.0000
4,zone1,20220919,1,OCARA,42,0,42,1.0000,0.0000


In [95]:
#Check if the captain was sourced from the exact same hex where the customer location is
df_all['captain_customer_same_hex'] = np.where(df_all['captain_location_hex_8'] == df_all['customer_location_hex_8'], 'sourced_from_same_hex',
 'sourced_from_different_hex')

#Check the number of orders serviced from the same hex
sourced_hex = pd.pivot_table(data = df_all, index = ['zone_name', 'yyyymmdd', 'hour', 'final_order_status'], columns = ['captain_customer_same_hex'], 
values = 'count', aggfunc='sum', fill_value=0).reset_index().rename_axis(None, axis=1)

#Create a column for total rides
sourced_hex['total'] = sourced_hex[['sourced_from_same_hex', 'sourced_from_different_hex']].sum(axis = 1)

#Calculate the percentage of each order_status
sourcing_hex = ['sourced_from_same_hex', 'sourced_from_different_hex']
for hex in sourcing_hex:
  sourced_hex['{}_%'.format(hex)] = round(sourced_hex[hex]/sourced_hex['total'], 4)
sourced_hex.head()

,zone_name,yyyymmdd,hour,final_order_status,sourced_from_different_hex,sourced_from_same_hex,total,sourced_from_same_hex_%,sourced_from_different_hex_%
0,zone1,20220919,0,COBRA,2,32,34,0.9412,0.0588
1,zone1,20220919,0,OCARA,24,48,72,0.6667,0.3333
2,zone1,20220919,0,dropped,156,17,173,0.0983,0.9017
3,zone1,20220919,1,COBRA,1,12,13,0.9231,0.0769
4,zone1,20220919,1,OCARA,8,34,42,0.8095,0.1905


In [96]:
#Check for the combinations for each customer-captain hex
df_dropped = df_all[df_all['final_order_status'] == 'dropped']
hex_combos = df_dropped.groupby(['customer_location_hex_8', 'captain_location_hex_8'])['count'].sum().reset_index()
hex_combos = hex_combos.sort_values(by = ['customer_location_hex_8', 'count'], ascending = False).reset_index(drop = True)

#Get the total orders done by customer  location
customer_orders = df_dropped.groupby(['customer_location_hex_8'])['count'].sum().reset_index()
customer_orders = customer_orders.rename(columns = {'count': 'total'})

#Merge customer orders with hex_combos
hex_combos = hex_combos.merge(customer_orders, on = 'customer_location_hex_8', how = 'left')
hex_combos['%_of_orders'] = round(hex_combos['count']/hex_combos['total'], 4)
hex_combos.head(10)

,customer_location_hex_8,captain_location_hex_8,count,total,%_of_orders
0,8860a259bdfffff,8860a259b9fffff,2214,22851,0.0969
1,8860a259bdfffff,8860a259b1fffff,2026,22851,0.0887
2,8860a259bdfffff,8860a25981fffff,1482,22851,0.0649
3,8860a259bdfffff,8860a259bdfffff,1372,22851,0.0600
4,8860a259bdfffff,8860a259bbfffff,1291,22851,0.0565
5,8860a259bdfffff,8860a25995fffff,953,22851,0.0417
6,8860a259bdfffff,8860a25987fffff,688,22851,0.0301
7,8860a259bdfffff,8860a25983fffff,600,22851,0.0263
8,8860a259bdfffff,8860a24b59fffff,510,22851,0.0223
9,8860a259bdfffff,8860a24a65fffff,453,22851,0.0198


In [97]:
#Check the percentile distribution for each type of order per day
percentile_FM = df_all.groupby(['zone_name', 'final_order_status', 'yyyymmdd',
 'hour'])['accept_to_pickup_distance'].describe(percentiles = [0.25, 0.5, 0.75, 0.9, 0.99]).reset_index()
percentile_FM = percentile_FM.fillna(0)
percentile_FM.head()

,zone_name,final_order_status,yyyymmdd,hour,count,mean,std,min,25%,50%,75%,90%,99%,max
0,zone1,COBRA,20220919,0,15.0,0.722800,0.761799,0.011,0.17100,0.3800,1.09550,1.8162,2.33240,2.345
1,zone1,COBRA,20220919,1,8.0,0.790625,0.855547,0.000,0.14250,0.3525,1.64025,1.8309,2.03439,2.057
2,zone1,COBRA,20220919,2,4.0,1.046500,1.076666,0.001,0.22900,0.9500,1.76750,2.0780,2.26430,2.285
3,zone1,COBRA,20220919,3,1.0,0.144000,0.000000,0.144,0.14400,0.1440,0.14400,0.1440,0.14400,0.144
4,zone1,COBRA,20220919,4,2.0,1.184500,0.461741,0.858,1.02125,1.1845,1.34775,1.4457,1.50447,1.511


In [98]:
#Save all the datasets to file
sourced_zone.to_csv('hyderabad/sourcing_from_batch.csv', index = False)
sourced_hex.to_csv('hyderabad/sourcing_from_hex.csv', index = False)
hex_combos.to_csv('hyderabad/hex_combos.csv', index = False)
percentile_FM.to_csv('hyderabad/fm_percentile_dist.csv', index = False)

In [90]:
x = df_all[df_all['order_status'] == 'dropped']
x = df_all[df_all['customer_location_hex_8'] == '8860a24a29fffff']
x = x[x['captain_location_hex_8'] == '8860a251c1fffff']

In [91]:
x

,order_id,accept_to_pickup_distance,captain_location_hex_8,captain_location_latitude,captain_location_longitude,city_name,customer_location_hex_8,customer_location_latitude,customer_location_longitude,hhmmss,hour,quarter_hour,service_obj_service_name,time_bucket,weekday,yyyymmdd,order_status,cancel_reason,zone_name,count,new_cancel_reason,final_order_status,captain_customer_same_area,captain_customer_same_hex
903173,63331930b4e0423285b10dd0,0.777,8860a251c1fffff,17.262693,78.386894,Hyderabad,8860a24a29fffff,17.449446,78.348495,210928,21,2100,Link,21:00,2,20220927,dropped,NaN,zone1,1,no_cancellation,dropped,sourced from outside batching area,sourced from different hex
